In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import glob
import re
import json
import os.path
from urllib.parse import urlparse

In [2]:
inlinks = {}
file_path = "inlinks.json"
with open(file_path, "r") as f:
    inlinks = json.load(f)


In [3]:
# file_path = "inlinks_priya.json"
# with open(file_path, "r") as f:
#     data = json.load(f)
#     for link in data:
#         if link in inlinks:
#             inlinks[link] = list(set( inlinks[link] + data[link]))
#         else:
#             inlinks[link] = data[link]


In [4]:
len(inlinks)

40705

In [5]:
mapping = {
    "settings" : {
        "index": {
          "store": {
            "type": "fs"
          },
          "number_of_shards": 2,
          "number_of_replicas": 1
        },
        "analysis": {
          "analyzer": {
            "my_english": { 
              "type": "english",
              "stopwords_path": "stoplist.txt" 
            }
          }
        }
    },
    "mappings": {
        "doc": { 
          "properties": {
            "DOCNO": { "type" : "text"  }, 
            "TITLE": { "type" : "text"},
            "TEXT": { 
                "type": "text",
                "store": True,
                "fielddata": True,
                "term_vector": "with_positions_offsets_payloads",
                "analyzer": "my_english"
            }, 
            "HTML": { "type": "text"  },
            "INLINK" : { "type": "text"  },
            "OUTLINK": { "type": "text"  },
            "AUTHOR": {"type" : "text"},
            "DEPTH" : {"type":"integer"}

          }
        }
    }
}

In [6]:
es = Elasticsearch()
es.indices.create('crawled_data_final_new', body=mapping, ignore = 400)

{'acknowledged': True, 'shards_acknowledged': True}

In [7]:
import json

In [8]:
pro_path = "files/"

In [9]:
c = 0
for i in range(1, 21001):
    file_path = pro_path + str(i) + ".json"
    with open(file_path, "r") as f:
        data = json.load(f)
        data["INLINK"] = []
        data["AUTHOR"] = ["Parth"]

        
        if data["DOCNO"] in inlinks:
            data["INLINK"] += inlinks[data["DOCNO"]]
            
        
        res = es.get(index = "crawled_data_final_new", 
             doc_type = "doc",
             id = data['DOCNO'],
             ignore = 404)
        
        
        if res["found"] == True:
            if(res["_source"]["TEXT"] == " "):
                data["TEXT"] = " "
            
            data["INLINK"] = list(set( res["_source"]["INLINK"] + data["INLINK"]))
            data["OUTLINK"] = list(set( res["_source"]["OUTLINK"] + data["OUTLINK"]))
            data["AUTHOR"] =  list(set( res["_source"]["AUTHOR"] + data["AUTHOR"]))
        
            
        res = es.index(index="crawled_data_final_new", doc_type='doc', id=data['DOCNO'], body=data)


#         break
# #         break
# #         parsedUrl = urlparse(data["DOCNO"].lower())
